In [26]:
import gym, os
from itertools import count
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical, Multinomial
import torchvision.transforms as T
from wrappers import *
from torch.autograd import Variable

import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from PIL import Image
import statistics



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("Pong-v0")
env = make_env(env)

state_space = env.observation_space.shape[0]
print(env.observation_space.shape)
action_space = env.action_space.n
print(action_space)

print(device)

(84, 84, 4)
6
cpu


Helper function

In [27]:
resize = T.Compose([T.ToPILImage(),
                    T.Resize(40, interpolation=Image.CUBIC),
                    T.ToTensor()])

EPS_START = 1.0
EPS_END = 0.01
EPS_DECAY = 3000

def compute_returns(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def process_rollout(gamma = 0.99, lambd = 1.0, num_workers = 1):
    _, _, _, _, last_values = steps[-1]
    returns = last_values.data

    advantages = torch.zeros(num_workers, 1)
    #if cuda: advantages = advantages.cuda()

    out = [None] * (len(steps) - 1)

    # run Generalized Advantage Estimation, calculate returns, advantages
    for t in reversed(range(len(steps) - 1)):
        rewards, masks, actions, policies, values = steps[t]
        _, _, _, _, next_values = steps[t + 1]

        returns = rewards + returns * gamma * masks

        deltas = rewards + next_values.data * gamma * masks - values.data
        advantages = advantages * gamma * lambd * masks + deltas

        out[t] = actions, policies, values, returns, advantages

    # return data as batched Tensors, Variables
    return map(lambda x: torch.cat(x, 0), zip(*out))

def get_screen(x):
    state = np.array(x)
    state = state.transpose((2, 0, 1))
    state = torch.from_numpy(state)
    state = state.float() / 255
    return state.unsqueeze(0)

def ortho_weights(shape, scale=1.):
    """ PyTorch port of ortho_init from baselines.a2c.utils """
    shape = tuple(shape)

    if len(shape) == 2:
        flat_shape = shape[1], shape[0]
    elif len(shape) == 4:
        flat_shape = (np.prod(shape[1:]), shape[0])
    else:
        raise NotImplementedError

    a = np.random.normal(0., 1., flat_shape)
    u, _, v = np.linalg.svd(a, full_matrices=False)
    q = u if u.shape == flat_shape else v
    q = q.transpose().copy().reshape(shape)

    if len(shape) == 2:
        return torch.from_numpy((scale * q).astype(np.float32))
    if len(shape) == 4:
        return torch.from_numpy((scale * q[:, :shape[1], :shape[2]]).astype(np.float32))

def game_initializer(module):
    """ Parameter initializer for Atari models
    Initializes Linear, Conv2d, and LSTM weights.
    """
    classname = module.__class__.__name__

    if classname == 'Linear':
        module.weight.data = ortho_weights(module.weight.data.size(), scale=np.sqrt(2.))
        module.bias.data.zero_()

    elif classname == 'Conv2d':
        module.weight.data = ortho_weights(module.weight.data.size(), scale=np.sqrt(2.))
        module.bias.data.zero_()

    elif classname == 'LSTM':
        for name, param in module.named_parameters():
            if 'weight_ih' in name:
                param.data = ortho_weights(param.data.size(), scale=1.)
            if 'weight_hh' in name:
                param.data = ortho_weights(param.data.size(), scale=1.)
            if 'bias' in name:
                param.data.zero_()

Creating two seperate networks the actor and the critic

In [28]:
end_out = 64


class ActorCritic(nn.Module):
    def __init__(self, num_frames, outputs):
        super(ActorCritic, self).__init__()
        self.conv1 = nn.Conv2d(num_frames, 32, kernel_size=8, stride=4)
        # self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        # self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, end_out, kernel_size=3, stride=1)
        
        self.down = nn.Linear(7*7*64, 512)
        self.a = nn.Linear(512, outputs)
        self.c = nn.Linear(512, 1)
        
        self.apply(game_initializer)
        self.a.weight.data = ortho_weights(self.a.weight.size(), scale=.01)
        self.c.weight.data = ortho_weights(self.c.weight.size())

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.down(x.view(x.size(0), -1))
        a = self.a(x)
        c = self.c(x)
        return a, c

In [29]:
learn_rate = 1e-4
n_iters = 500000
avg_over = 50
val_coeff = 0.5
ent_coeff = 0.01

#init_screen = get_screen(env.reset())
#_, num_frames, screen_height, screen_width = init_screen.shape

# Get number of actions from gym action space
#n_actions = env.action_space.n

#actor_critic = ActorCritic(num_frames, n_actions).to(device)
#critic = Critic(num_frames, screen_height, screen_width, n_actions).to(device)

#optimizer = optim.Adam(actor_critic.parameters(), lr=learn_rate)
#optimizerC = optim.Adam(critic.parameters(), lr=learn_rate)

#score_avg = []
#ac_loss_list = []
#score_avg_all = []
#ac_loss_list_all = []


for iter in range(n_iters):
    state = get_screen(env.reset())
    #state = Variable(state)
    #log_probs = []
    steps = []
    #values = []
    #rewards = []
    #masks = []
    entropy = 0
    reward_sum = 0

    for i in count():
        #env.render()
        #state = torch.FloatTensor(state).to(device)
        logit, values = actor_critic(state)
        
        probs = F.softmax(logit)
        actions = probs.multinomial(1).data
        
        state, rewards, dones, _ = env.step(actions.cpu().numpy())
        reward_sum += rewards
        #masks = (1. - torch.from_numpy(np.array(dones, dtype=np.float32)))
        #rewards = torch.from_numpy(rewards)
        #rewards = torch.from_numpy(np.array(rewards, dtype=np.float32))
        rewards = (torch.tensor([rewards], dtype=torch.float, device=device)).unsqueeze(-1)
        masks = (torch.tensor([1-dones], dtype=torch.float, device=device)).unsqueeze(-1)
        
        
        state = get_screen(state)
        #state = Variable(state)
        
        #reward_sum += rewards
        
        steps.append((rewards, masks, actions, logit, values))
        if dones:
            #print('Iteration: {}, Score: {}'.format(iter, i))
            score_avg.append(reward_sum)
            reward_sum=0
            break
    
        
    
    #next_state = torch.FloatTensor(next_state).to(device)
    _, final_values = actor_critic(state)
    steps.append((None, None, None, None, final_values))
    actions, logit, values, returns, advantages = process_rollout()
   
    probs = F.softmax(logit)
    log_probs = F.log_softmax(logit)
    log_action_probs = log_probs.gather(1, actions)

    
    policy_loss = (-log_action_probs * advantages).sum()
    value_loss = (0.5*(values - returns) ** 2.).sum()
    entropy_loss = (log_probs * probs).sum()

    ac_loss = policy_loss + value_loss * val_coeff + entropy_loss * ent_coeff

    optimizer.zero_grad()
    ac_loss.backward()
    nn.utils.clip_grad_norm(actor_critic.parameters(), 40.)
    optimizer.step()
    
    ac_loss_list.append(ac_loss)
    #optimizer.zero_grad()
    if iter % avg_over == 0 and iter != 0:
        score_sum = 0
        loss_sum = 0
        for i in range(len(score_avg)):
            score_sum += score_avg[i]
            loss_sum += ac_loss_list[i]
        s = score_sum/avg_over
        l = loss_sum/avg_over
        print('Iteration: {}, Score avg: {}, Loss: {}'.format(iter+8750, s, l))
        score_avg_all.append(s)
        ac_loss_list_all.append(l)
        score_avg.clear()
        ac_loss_list.clear()

              
    

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:72: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:73: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:85: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Iteration: 50, Score avg: -20.96, Loss: 282.3039855957031
Iteration: 100, Score avg: -20.38, Loss: 467.82391357421875
Iteration: 150, Score avg: -20.42, Loss: 394.04461669921875
Iteration: 200, Score avg: -20.52, Loss: 304.4825439453125
Iteration: 250, Score avg: -20.38, Loss: 238.1055145263672
Iteration: 300, Score avg: -20.42, Loss: 227.94454956054688
Iteration: 350, Score avg: -20.32, Loss: 196.4358367919922
Iteration: 400, Score avg: -20.52, Loss: 252.54867553710938
Iteration: 450, Score avg: -20.4, Loss: 182.66552734375
Iteration: 500, Score avg: -20.44, Loss: 201.83441162109375
Iteration: 550, Score avg: -20.36, Loss: 238.1215362548828
Iteration: 600, Score avg: -20.5, Loss: 142.74374389648438
Iteration: 650, Score avg: -20.56, Loss: 59.531829833984375
Iteration: 700, Score avg: -20.42, Loss: 140.17816162109375
Iteration: 750, Score avg: -20.44, Loss: 184.8663330078125
Iteration: 800, Score avg: -20.58, Loss: 253.32696533203125
Iteration: 850, Score avg: -20.36, Loss: 184.8706817

Iteration: 6900, Score avg: -20.64, Loss: 123.48957061767578
Iteration: 6950, Score avg: -20.58, Loss: 165.39682006835938
Iteration: 7000, Score avg: -20.44, Loss: 136.24737548828125
Iteration: 7050, Score avg: -20.52, Loss: 137.9613800048828
Iteration: 7100, Score avg: -20.6, Loss: 58.385353088378906
Iteration: 7150, Score avg: -20.54, Loss: 113.27423858642578
Iteration: 7200, Score avg: -20.56, Loss: 124.04861450195312
Iteration: 7250, Score avg: -20.54, Loss: 68.62422180175781
Iteration: 7300, Score avg: -20.34, Loss: 133.56907653808594
Iteration: 7350, Score avg: -20.4, Loss: 173.4021453857422
Iteration: 7400, Score avg: -20.52, Loss: 117.35369873046875
Iteration: 7450, Score avg: -20.42, Loss: 141.75218200683594
Iteration: 7500, Score avg: -20.54, Loss: 143.6628875732422
Iteration: 7550, Score avg: -20.62, Loss: 134.37644958496094
Iteration: 7600, Score avg: -20.4, Loss: 195.7388458251953
Iteration: 7650, Score avg: -20.62, Loss: 115.85310363769531
Iteration: 7700, Score avg: -20.

KeyboardInterrupt: 

In [ ]:
torch.save(actor_critic, 'results/actor.pkl')
plt.plot(score_avg)

In [ ]:
plt.plot(ac_loss_list)

In [10]:
n_iter = 5
for iter in range(n_iter):
    state = get_screen(env.reset())
    log_probs = []
    values = []
    rewards = []
    masks = []
    entropy = 0
    env.reset()
    reward_sum = 0

    for i in count():
        env.render()
        logit, values = actor_critic(state)
        
        probs = F.softmax(logit)
        actions = probs.multinomial(1).data
        
        state, rewards, dones, _ = env.step(actions.cpu().numpy())
        reward_sum += rewards
        masks = (1. - torch.from_numpy(np.array(dones, dtype=np.float32))).unsqueeze(-1)
        #rewards = torch.from_numpy(rewards)
        #rewards = torch.from_numpy(np.array(rewards, dtype=np.float32)).unsqueeze(-1)
        #rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
        #masks.append(torch.tensor([1-done], dtype=torch.float, device=device))
        
        
        state = get_screen(state)
        #state = Variable(state)
        
        #reward_sum += rewards
        
        steps.append((rewards, masks, actions, logit, values))
        if done:
            break
env.close()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


NameError: name 'done' is not defined

In [ ]:
torch.save(actor, 'results/actor.pkl')
torch.save(critic, 'results/critic.pkl')
env.close()